In [20]:
import matplotlib.pyplot as plt
import os
import pandas as pd

In [21]:
def create_image_paths(images_ids):
    images_paths = []
    img = plt.imread('C:/Users/mahmo/Desktop/Leaf-Classification/input/images/1.jpg')
    source_folder = 'C:/Users/mahmo/Desktop/Leaf-Classification/input/images/'
    for id in images_ids:
        img_path = source_folder + str(id) + '.jpg'
        images_paths.append(img_path)
    return images_paths

In [22]:
def draw_some_images(images_paths):
    fig , axis = plt.subplots(2,4)
    fig.set_figheight(10)
    fig.set_figwidth(20)
    # load images
    images = []
    for path in images_paths:
        images.append(plt.imread(path))
    i , j = 0 , 0
    for img in images:
        axis[i,j].imshow(img,cmap='gray')
        j = j + 1
        if j == 4:
            i = i + 1
            j = 0

In [23]:
def read_data(file_name):
    # read data from csv file in input folder
    cur_dir = os.getcwd()
    file_path = 'C:/Users/mahmo/Desktop/Leaf-Classification/input/' + file_name
    data = pd.read_csv(os.path.join(cur_dir,file_path))
    return data